In [1]:

from langchain.chat_models import ChatOpenAI
# 테스트 파일,파워포인트, html, pdfs, images , 등등 로드 할 수 있다.
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
#임베딩 캐시하기
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough,RunnableLambda

llm = ChatOpenAI(
    temperature=0.1,
)

cache_dir = LocalFileStore("./.cache/")

cache_embeddings = CacheBackedEmbeddings

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap= 100,
)

loader = UnstructuredFileLoader("./files/chapter_one.pdf")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cache_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings,cache_dir)

vectorstore = FAISS.from_documents(docs, cache_embeddings)

retriver = vectorstore.as_retriever()


map_docs_prompt =ChatPromptTemplate.from_messages([
            (
            "system",
            """
            Use the following portion of a long document to see if any of the text is relevant to answer the question. Return any relevant text verbatim. If there is no relevant text, return : ''
            -------
            {context}
            """,
        ),
        ("human", "{question}"),

])

map_docs_chain = map_docs_prompt | llm

"""
아래로 표현가능
def map_docs(inputs):
    documents = inputs['documents']
    question = inputs['question']
    results= []
    for document in documents:
        result =map_docs_chain.invoke({
            "context" : document.page_content,
            "question" : question
        }).content
        results.append(result)
    results = "\n\n".join(results)
    return results
"""
# documents의 모든 documnet에 대한 chain 실행의 결과를 저장한 List를 만들고 있다. 작업의 결과는 list 이고 그걸 join에 전달하고있다.
def map_docs(inputs):
    documents = inputs['documents']
    question = inputs['question']
    return "\n\n".join(
        map_docs_chain.invoke(
            {"context" : doc.page_content,
            "question" : question}
        ).content
        for doc in documents
    )

final_prompt = ChatPromptTemplate.from_messages([
        (
            "system",
            """
            Given the following extracted parts of a long document and a question, create a final answer. 
            If you don't know the answer, just say that you don't know. Don't try to make up an answer.
            ------
            {context}
            """,
        ),
        ("human", "{question}"),

])

#document 필요,
map_chain = {
    "documents" : retriver ,
    "question" : RunnablePassthrough(),
    }| RunnableLambda(map_docs)

chain =  {"context": map_chain,"question" : RunnablePassthrough() } | final_prompt|llm

chain.invoke("How many ministries are mentioned")

AIMessage(content='Three ministries are mentioned in the given text: the Ministry of Love, the Ministry of Plenty, and the Ministry of Truth.')